<a href="https://colab.research.google.com/github/04pys/cs-systems-labs/blob/main/notebooks/phase01_lowlevel_basics/lab02_tiling_blocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%bash
cat > main.cpp << 'CPP'
#include <bits/stdc++.h>
using namespace std;

static inline long long now_ns() {
  return chrono::duration_cast<chrono::nanoseconds>(
    chrono::steady_clock::now().time_since_epoch()
  ).count();
}

int main() {
  ios::sync_with_stdio(false);
  cin.tie(nullptr);

  int N = 8192;                 // 필요 시 2048, 4096, 8192로 변경 가능함
  vector<int> a((long long)N * N, 1);

  auto bench_rowcol = [&](const string& name, bool row_major) {
    volatile long long sum = 0;
    long long t0 = now_ns();

    if (row_major) {
      for (int i = 0; i < N; i++)
        for (int j = 0; j < N; j++)
          sum += a[(long long)i * N + j];
    } else {
      for (int j = 0; j < N; j++)
        for (int i = 0; i < N; i++)
          sum += a[(long long)i * N + j];
    }

    long long t1 = now_ns();
    double ms = (t1 - t0) / 1e6;
    cout << name << " N=" << N << " time_ms=" << ms << " sum=" << sum << "\n";
  };

  auto bench_tiled_col = [&](int B) {
    volatile long long sum = 0;
    long long t0 = now_ns();

    // col-major-like( j outside, i inside )를 유지하되
    // (i, j)를 BxB 블록으로 쪼개서 작은 작업집합으로 처리하는 구조임
    for (int jj = 0; jj < N; jj += B) {
      int j_end = min(jj + B, N);
      for (int ii = 0; ii < N; ii += B) {
        int i_end = min(ii + B, N);
        for (int j = jj; j < j_end; ++j) {
          for (int i = ii; i < i_end; ++i) {
            sum += a[(long long)i * N + j];
          }
        }
      }
    }

    long long t1 = now_ns();
    double ms = (t1 - t0) / 1e6;
    cout << "tiled_col"
         << " N=" << N
         << " B=" << B
         << " time_ms=" << ms
         << " sum=" << sum << "\n";
  };

  // 워밍업
  bench_rowcol("warmup_row", true);

  // baseline
  bench_rowcol("row_major", true);
  bench_rowcol("col_major_like", false);

  // tiled variants
  for (int B : {1,2,4,8, 16, 32, 64, 128, 256}) {
    bench_tiled_col(B);
  }

  return 0;
}

CPP

In [14]:
!g++ -O2 -std=c++17 main.cpp -o main

In [12]:
!./main

warmup_row N=4096 time_ms=36.5356 sum=16777216
row_major N=4096 time_ms=41.4764 sum=16777216
col_major_like N=4096 time_ms=234.087 sum=16777216
tiled_col N=4096 B=1 time_ms=494.682 sum=16777216
tiled_col N=4096 B=2 time_ms=173.691 sum=16777216
tiled_col N=4096 B=4 time_ms=155.983 sum=16777216
tiled_col N=4096 B=8 time_ms=110.584 sum=16777216
tiled_col N=4096 B=16 time_ms=83.4921 sum=16777216
tiled_col N=4096 B=32 time_ms=76.3236 sum=16777216
tiled_col N=4096 B=64 time_ms=81.9885 sum=16777216
tiled_col N=4096 B=128 time_ms=83.9152 sum=16777216
tiled_col N=4096 B=256 time_ms=87.8456 sum=16777216


In [15]:
!./main

warmup_row N=8192 time_ms=141.795 sum=67108864
row_major N=8192 time_ms=144.349 sum=67108864
col_major_like N=8192 time_ms=935.589 sum=67108864
tiled_col N=8192 B=1 time_ms=2333.78 sum=67108864
tiled_col N=8192 B=2 time_ms=1143.54 sum=67108864
tiled_col N=8192 B=4 time_ms=778.164 sum=67108864
tiled_col N=8192 B=8 time_ms=499.459 sum=67108864
tiled_col N=8192 B=16 time_ms=389.186 sum=67108864
tiled_col N=8192 B=32 time_ms=348.018 sum=67108864
tiled_col N=8192 B=64 time_ms=569.496 sum=67108864
tiled_col N=8192 B=128 time_ms=558.668 sum=67108864
tiled_col N=8192 B=256 time_ms=533.197 sum=67108864
